# Core concepts

## Predefined atlases, parcellations, and reference spaces

The `siibra.core` module provides classes for the core atlas concepts. These include 

 - the `Atlas` class as the basic entry point for working with a brain atlas 
 - the `Parcellation` class, defining a particular brain parcellation scheme
 - the `Space` class, defining a particular reference space
 
The above classes are all derived from the basic `AtlasConcept` class, and represent semantic concepts. When first loading a new siibra version, `siibra` automatically builds a registry with predefined objects for each of these classes. The configuration information is retrieved from a versioned online repository that we maintain with siibra. 

In [ ]:
!pip install siibra==0.4a29


In [ ]:
import siibra
assert siibra.__version__ >= "0.4a27"
import os
import matplotlib
%matplotlib notebook

To performing this bootstrap process, an internet connection is required to import siibra the first time. However, the configuration information is then stored in a local cache folder on your system, so after the first time you will be able to import your `siibra` package without being online.

## Accessing predefined atlases

Siibra provides the preconfigured instances of the core concepts via the "instance tables" `siibra.atlases`, `siibra.parcellations`, and `siibra.spaces`. Elements in each table can be accessed in different ways:

 - iterate over all instances
 - by fuzzy matching of keyword or name with the index operator
 - by tab-completion

Let's try this out for the `siibra.atlases` registry.

In [ ]:
# Which atlases are provided? We can iterate over objects in the registry.
for atlas in siibra.atlases:
    print(repr(atlas), "-", atlas.name)

In [ ]:
# Access the first element in the registry
siibra.atlases[0]

In [ ]:
# Access elements by their name as attributes
# with autocompletion by most Python interpreters
siibra.atlases.MULTILEVEL_HUMAN_ATLAS

Note how both alternatives provide the same object. The easiest and recommended way to access items from a registry however, is to use keywords for accessing elements. `siibra` will try to figure out the matching item, or inform you if you need to be more precise.

In [ ]:
# The easiest way: Using string matching of keywords
siibra.atlases['human']

## Accessing parcellations and regions

While the registry `siibra.parcellations` gives access to all available parcellations, the recommended way is to access parcellations via an atlas object. Each atlas object provices a filtered registry of parcellations.

In [ ]:
atlas = siibra.atlases['human']
list(atlas.parcellations)

We can get access to any parcellation object using `atlas.get_parcellation`, or of course by directly using the registry:

In [ ]:
julichbrain = atlas.get_parcellation('julich')  # will return the latest version per default

In [ ]:
# let's look at some metadata
print(f"Name:     {julichbrain.name}")
print(f"Id:       {julichbrain.id}")
print(f"Modality: {julichbrain.modality}\n")
print(f"{julichbrain.description}\n")
for p in julichbrain.publications:
    print(p['citation'])

The resulting parcellation is a semantic object. It does not itself represent a map, but rather the defintion of the parcellation. Thus it provides

- the region hierarchy
- functions to find and access regions
- functions to access different forms of maps

Let's search a region.

In [ ]:
# search regions known by the parcellation
for region in julichbrain.find_regions('amygdala'):
    print(region.name)

As you see, areas often appear three times: Julich-Brain defines them separately for the left and right hemisphere, and additionally defines a common parent region. In fact the parent object represents the corresponding subtree. We can more easily access individual regions by using `get_region` instead of `find_regions`. This method assumes the region specification is unique, and returns a single region object or fails.

In [ ]:
# the whole amygdala subtree
julichbrain.get_region('amygdala')

You may output the subtree anchored at a given region, if any, using `Region.tree2str()`. This is useful to inspect a region object.

In [ ]:
amygdala = julichbrain.get_region('amygdala')
print(amygdala.tree2str())

In [ ]:
# only VTM, a small subtree
vtm = julichbrain.get_region('VTM')
print(vtm.tree2str())

In [ ]:
# the exact left VTM is not a tree, it is a leaf of the region hierarchy
vtm_l = julichbrain.get_region('vtm left')
print(vtm_l.tree2str())

To search for regions, we need not explicitly fetch the parcellation object. The atlas object provides a similar function, however, it will return matching regions from all parcellations it knows.

In [ ]:
# search all regions known by the atlas
for r in atlas.find_regions('amygdala'):
    print(f"{r.name:30.30} {r.parcellation}")

## Accessing maps

Different from the above semantic objects - atlases, spaces, parcellations, regions - a parcellation map or region map is a spatial object. In some atlases, parcellations are defined in different spaces, so `siibra` expects you to specify the space when accessing the map of a parcellation. Let's access the maximum probability map of Julich-Brain in the MNI152 space to see how that works.

Note that when running this for the first time, `siibra` will download the data from EBRAINS, which takes a bit. the parcellation map is then stored in a cache directory on your computer, so future calls are fast.

In [ ]:
mpm = julichbrain.get_map(space=siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
mpm

### Labelled parcellation maps

The returned map still does not contain the actual image data, only the information about the image resources for each brain region. This is because `siibra` uses a lazy strategy for loading data. To access the actual image data, we call the `fetch()` method. We use the wonderful `nilearn` library for plotting the map. It plots in the MNI152 space by default, so as long as we work in this space plotting is simple enough.

Some parcellations (and other 3D volumes) are split into multiple fragments represented in separate image volumes. For Julich-Brain 2.9, each hemisphere is in a different fragment. We can fetch individual fragments, but if no fragment is specified, siibra will merge the available ones into a single volume:

In [ ]:
from nilearn import plotting
plotting.plot_stat_map(mpm.fetch(), title=f"{mpm.parcellation.name} (merged fragments)")

To fetch only the left hemisphere, we simply specifiy the fragment in the `fetch()` call:

In [ ]:
plotting.plot_stat_map(mpm.fetch(fragment='left'), title=mpm.parcellation.name+" (left)")

### Probabilistic maps

Julich-Brain, like some other parcellations, is a probabilistic parcellation. The labelled volumes in the maximum probability map (mpm) above are just a simplified representation, displaying for each voxel the brain region of highest probability. We can access the much richer information of the probability maps, which provide statistical information in the reference space for each parcticular region.

We received the labelled volumes above because `siibra` uses labelled volumes as the default map type. To retrieve probability maps, we explicitly select `siibra.MapType.STATISTICAL`.

In [ ]:
pmaps = julichbrain.get_map(
    space=siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC,
    maptype=siibra.MapType.STATISTICAL
)

In [ ]:
pmaps.fetch_iter()

We can iterate over all >300 probability maps using the `fetch_iter()` method, or fetch individual probability maps by their index, region name, or using a region object. We recommend avoiding to deal with label and volume indices - it is one of `siibra`'s strengths to hide this complexity from you.

In [ ]:
pmap = pmaps.fetch(region='hoc5 right')
plotting.plot_stat_map(pmap, title=f'hOc5 right of {pmaps.parcellation.name}')

Of course we can easily access the indices of this region's map if we like to. Since the region is a complete volume in the statistical map, it has no label or fragment associated in the probability maps:

In [ ]:
index = pmaps.get_index(region='hoc5 right')
index

Note that this is different if we request the index of the same region in the maximum probability maps! Since these are a labelled parcellation and not a statistical one, the same region is there represented by a label index in one fragment of the image volume:

In [ ]:
index = mpm.get_index(region='hoc5 right')
index

As mentioned before, while not recommended, we can also use this index to fetch from the map instead of using a region or region name:

In [ ]:
index = pmaps.get_index(region='hoc5 right')
pmap = pmaps.fetch(index=index)
plotting.plot_stat_map(pmap, title=f'hOc5 right of {pmaps.parcellation.name}')

If we fetch a particular region or region index from the maximum probability map, `siibra` will construct a binary mask of the region from the labelled volume. Note the corresponding location, but significant difference in shape!

In [ ]:
mask = mpm.fetch(region='hoc5 right')
plotting.plot_roi(mask, title=f'hOc5 right of {pmaps.parcellation.name}')

## Extracting volumes of interest from high-resolution data

Accessing image volumes is at the heart of `siibra`, and also works for high resolution images such as the BigBrain model. The BigBrain can be found and accessed as the template of the BigBrain reference space. However, fetching BigBrain at full resolution is not a good idea - it is a 1TByte dataset. `siibra` will therefore by default fetch a downscaled version:

In [ ]:
bigbrain = siibra.spaces.BIG_BRAIN_HISTOLOGY.get_template()
bigbrain_img = bigbrain.fetch()
plotting.plot_img(bigbrain_img, cmap='gray')

If we request the full resolution, `siibra` will complain and choose a larger but feasible resolution.

In [ ]:
bigbrain.fetch(resolution_mm=0.02)

To work with full resolution data, we typically fetch volumes of interest only (VOIs). These can be defined in different ways - by specifying corner points, or extracting the bounding box of a region.

In [ ]:
voi = siibra.locations.BoundingBox(
    (-3.979, -61.256, 3.906),
    (5.863, -55.356, -2.487),
    space=siibra.spaces.BIG_BRAIN_HISTOLOGY
)

Now we can extract a chunk from the BigBrain template a full resolution of 20 micron using this volume of interest.

In [ ]:
bigbrainchunk = bigbrain.fetch(resolution_mm=0.02, voi=voi)
plotting.view_img(bigbrainchunk, bg_img=None, cmap='gray')

The resulting image chunk sits properly in its reference space, so we can also plot it in anatomical context of the low-resolution whole brain image that we fetched already above.

In [ ]:
plotting.plot_roi(bigbrainchunk, bg_img=bigbrain_img)

We can apply this volume of interest to extract chunks from other objects in the same space, like parcellation maps. Here we use the coritcal layer maps of BigBrain. We can use the LabelledParcellation object for the cortical layer maps that we requested further above, but no call its `fetch()` method again with a different resolution and the volume of interest specification.

In [ ]:
layermap = siibra.parcellations.get('cortical layers').get_map(space='bigbrain')
mask = layermap.fetch(resolution_mm=0.16,voi=voi)
plotting.view_img(mask, bg_img=bigbrainchunk, opacity=.1, symmetric_cmap=False)

As mentioned above, we can easily define such volumes of interest by requesting the bounding box of a region object in a desired space. While brain regions are not always mapped in the desired space, siibra will automatically warp the locations acoordingly for you. Here we request a bounding box for area 44 left as defined in Julich-Brain. `siibra` realizes that there is no map for this region in BigBrain yet, so it uses the map in MNI space to compute the bounding box, and then warps it to BigBrain:

In [ ]:
area44l = julichbrain.get_region('44 left')
voi = area44l.get_bounding_box(space=siibra.spaces.BIG_BRAIN_HISTOLOGY)

We will not fetch data now, but we can compute the volume of the bounding box:

In [ ]:
voi.volume